Data Science Capstone Project Notebook

This notebook will be used to complete the final project in the IBM Data Science course

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
import requests
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
print("All modules installed and imported")

     |████████████████████████████████| 98 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 4.1 MB/s  eta 0:00:01
All modules installed and imported


The csv that will be used was uploaded to Watson Studio and is imported here

In [2]:
# The code was removed by Watson Studio for sharing.

The csv can now be converted to a dataframe, which we shall process and clean by removing unwated data and reorganising the dataframe to be more easily interpreted 

In [3]:
df = pd.read_csv(body)
df.head()

,Postcode,In Use?,Latitude,Longitude,County,District,London zone,Nearest station,Postcode area,Postcode district,Average Income
0,BR1 1AA,Yes,51.401546,0.015415,Greater London,Bromley,5,Bromley South,BR,BR1,63100
1,BR1 1AB,Yes,51.406333,0.015208,Greater London,Bromley,4,Bromley North,BR,BR1,56100
2,BR1 1AD,No,51.400057,0.016715,Greater London,Bromley,5,Bromley South,BR,BR1,63100
3,BR1 1AE,Yes,51.404543,0.014195,Greater London,Bromley,4,Bromley North,BR,BR1,63100
4,BR1 1AF,Yes,51.401392,0.014948,Greater London,Bromley,5,Bromley South,BR,BR1,63100


In [4]:
df.shape

(323306, 11)

In [5]:
df = df[df["In Use?"].str.contains("Yes")]
df.head()

,Postcode,In Use?,Latitude,Longitude,County,District,London zone,Nearest station,Postcode area,Postcode district,Average Income
0,BR1 1AA,Yes,51.401546,0.015415,Greater London,Bromley,5,Bromley South,BR,BR1,63100
1,BR1 1AB,Yes,51.406333,0.015208,Greater London,Bromley,4,Bromley North,BR,BR1,56100
3,BR1 1AE,Yes,51.404543,0.014195,Greater London,Bromley,4,Bromley North,BR,BR1,63100
4,BR1 1AF,Yes,51.401392,0.014948,Greater London,Bromley,5,Bromley South,BR,BR1,63100
5,BR1 1AG,Yes,51.401392,0.014948,Greater London,Bromley,5,Bromley South,BR,BR1,63100


In [6]:
df = df[df["Postcode district"].str.contains("WC|EC")]
df.shape

(6321, 11)

In [7]:
df.drop(columns=["In Use?", "County", "London zone", "Nearest station", "Postcode area"], inplace = True)
df.head()

,Postcode,Latitude,Longitude,District,Postcode district,Average Income
56706,EC1A 1AA,51.525136,-0.111183,Islington,EC1A,53800
56708,EC1A 1AH,51.524565,-0.112042,Islington,EC1A,53800
56712,EC1A 1AZ,51.524503,-0.112088,Islington,EC1A,53800
56714,EC1A 1BB,51.524565,-0.112042,Islington,EC1A,53800
56720,EC1A 1DN,51.524565,-0.112042,Islington,EC1A,53800


In [8]:
district = df['Postcode district']
df.drop(columns = ["Postcode", "Postcode district"], inplace = True)
df.insert(0, 'Postcode District', district)
df.head()

,Postcode District,Latitude,Longitude,District,Average Income
56706,EC1A,51.525136,-0.111183,Islington,53800
56708,EC1A,51.524565,-0.112042,Islington,53800
56712,EC1A,51.524503,-0.112088,Islington,53800
56714,EC1A,51.524565,-0.112042,Islington,53800
56720,EC1A,51.524565,-0.112042,Islington,53800


In [9]:
df.shape

(6321, 5)

In [10]:
df2 = df.groupby(["Postcode District", "District"]).mean()
df2.head()

Latitude  Longitude  Average Income
Postcode District District                                            
EC1A              City of London  51.518124  -0.100295         65400.0
                  Islington       51.524156  -0.110065         53800.0
EC1M              Camden          51.520813  -0.106501         50000.0
                  City of London  51.520806  -0.098285         65400.0
                  Islington       51.521362  -0.102184         53800.0

In [11]:
df2.shape

(90, 3)

In [12]:
df2.reset_index(inplace = True)
print(df2.shape)
df2.head()

(90, 5)


,Postcode District,District,Latitude,Longitude,Average Income
0,EC1A,City of London,51.518124,-0.100295,65400.0
1,EC1A,Islington,51.524156,-0.110065,53800.0
2,EC1M,Camden,51.520813,-0.106501,50000.0
3,EC1M,City of London,51.520806,-0.098285,65400.0
4,EC1M,Islington,51.521362,-0.102184,53800.0


In [13]:
df2.shape

(90, 5)

Now we have our dataframe ready, we can start the analysis. First we can use folium to visulise the data. Using geopy to get to coordinates of Central London, we can plot our districts on a map

In [14]:
address = 'City of London, England'

geolocator = Nominatim(user_agent="Ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5156177, -0.0919983.


In [15]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=13)


for lat, lng, postcode, district in zip(df2['Latitude'], df2['Longitude'], df2['Postcode District'], df2["District"]):
    label = '{}, {}'.format(postcode, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Now we can use the FourSquare API to find the top ten venues in a radius around each district. First we have to define some functions that will send the request to Foursquare and then extract the useful data

In [16]:
# The code was removed by Watson Studio for sharing.

Foursquare credentials set and hidden


In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Postcode District", 
                  "District Latitude", 
                  "District Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [18]:
London_venues = getNearbyVenues(names=df2["Postcode District"],
                                   latitudes=df2["Latitude"],
                                   longitudes=df2["Longitude"]
                                  )
London_venues.groupby("Postcode District").count()

EC1A
EC1A
EC1M
EC1M
EC1M
EC1N
EC1N
EC1N
EC1P
EC1R
EC1R
EC1V
EC1V
EC1Y
EC1Y
EC2A
EC2A
EC2A
EC2M
EC2M
EC2M
EC2M
EC2N
EC2N
EC2P
EC2P
EC2R
EC2R
EC2V
EC2V
EC2Y
EC2Y
EC3A
EC3A
EC3M
EC3N
EC3N
EC3N
EC3P
EC3P
EC3R
EC3R
EC3V
EC3V
EC4A
EC4A
EC4A
EC4M
EC4M
EC4N
EC4N
EC4P
EC4R
EC4R
EC4V
EC4V
EC4Y
EC4Y
EC4Y
WC1A
WC1A
WC1B
WC1B
WC1E
WC1E
WC1H
WC1H
WC1N
WC1N
WC1R
WC1V
WC1V
WC1V
WC1X
WC1X
WC2A
WC2A
WC2A
WC2A
WC2B
WC2B
WC2B
WC2E
WC2H
WC2H
WC2H
WC2N
WC2N
WC2R
WC2R


,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode District,,,,,,
EC1A,148,148,148,148,148,148
EC1M,192,192,192,192,192,192
EC1N,245,245,245,245,245,245
EC1P,66,66,66,66,66,66
EC1R,179,179,179,179,179,179
EC1V,160,160,160,160,160,160
EC1Y,179,179,179,179,179,179
EC2A,266,266,266,266,266,266
EC2M,314,314,314,314,314,314


Now we have the data from FourSquare we can use onehot encoding to make one dataframe containing all our districts with thier top ten venues

In [19]:
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

London_onehot['Postcode District'] = London_venues['Postcode District'] 

fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]

Ln_grouped = London_onehot.groupby('Postcode District').mean().reset_index()
Ln_grouped.shape

(41, 222)

In [20]:
num_top_venues = 10

for district in Ln_grouped['Postcode District']:
    print("----"+district+"----")
    temp = Ln_grouped[Ln_grouped['Postcode District'] == district].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----EC1A----
                  venue  freq
0                   Pub  0.11
1           Coffee Shop  0.07
2    Italian Restaurant  0.05
3     French Restaurant  0.04
4                   Bar  0.03
5        Sandwich Place  0.03
6                  Café  0.03
7           Pizza Place  0.03
8  Gym / Fitness Center  0.03
9                 Hotel  0.03


----EC1M----
                   venue  freq
0                    Pub  0.08
1            Coffee Shop  0.07
2             Food Truck  0.07
3                  Hotel  0.06
4                   Café  0.05
5      French Restaurant  0.05
6   Gym / Fitness Center  0.05
7     Italian Restaurant  0.04
8  Vietnamese Restaurant  0.03
9               Wine Bar  0.03


----EC1N----
                  venue  freq
0                   Pub  0.11
1           Coffee Shop  0.07
2        Sandwich Place  0.05
3                 Hotel  0.04
4    Italian Restaurant  0.03
5          Cocktail Bar  0.03
6              Wine Bar  0.03
7            Food Truck  0.03
8           Pizz

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Postcode District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

London_venues_sorted = pd.DataFrame(columns=columns)
London_venues_sorted['Postcode District'] = Ln_grouped['Postcode District']

for ind in np.arange(Ln_grouped.shape[0]):
    London_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Ln_grouped.iloc[ind, :], num_top_venues)

London_venues_sorted

,Postcode District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EC1A,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Gym / Fitness Center,Café,Hotel,Bar,Pizza Place
1,EC1M,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
2,EC1N,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Falafel Restaurant,Sushi Restaurant
3,EC1P,Pub,Coffee Shop,Pizza Place,Cocktail Bar,Sandwich Place,Garden,Middle Eastern Restaurant,History Museum,Food Truck,Bar
4,EC1R,Pub,Coffee Shop,Café,Food Truck,Bar,Pizza Place,Vietnamese Restaurant,Middle Eastern Restaurant,Bakery,Sandwich Place
5,EC1V,Coffee Shop,Pub,Italian Restaurant,Café,Bar,Sandwich Place,Cocktail Bar,Vietnamese Restaurant,Food Truck,Gym / Fitness Center
6,EC1Y,Coffee Shop,Gym / Fitness Center,Food Truck,Italian Restaurant,Café,Pub,Concert Hall,Bar,Art Gallery,Sushi Restaurant
7,EC2A,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Italian Restaurant,Pub,Pizza Place,Bar,Café,Cocktail Bar
8,EC2M,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Bar,Café,Plaza,French Restaurant
9,EC2N,Pub,Coffee Shop,Hotel,Cocktail Bar,Sandwich Place,Gym / Fitness Center,Restaurant,Garden,Pizza Place,Indian Restaurant


With our dataframe created, we can use k means clustering to group the districts together into similar clusters. This will make analysis easier later as we can find the best district in each cluster and then compare them to one another

As k means is a random process, the clusters may be different each time, so may not match the final report if re run at a later date

In [23]:
kclusters = 5

Ln_grouped_clustering = Ln_grouped.drop('Postcode District', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ln_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 1, 4, 1, 1, 1], dtype=int32)

In [24]:
London_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Ln_merged = df2

Ln_merged = Ln_merged.join(London_venues_sorted.set_index('Postcode District'), on='Postcode District')

Ln_merged

,Postcode District,District,Latitude,Longitude,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EC1A,City of London,51.518124,-0.100295,65400.000000,0,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Gym / Fitness Center,Café,Hotel,Bar,Pizza Place
1,EC1A,Islington,51.524156,-0.110065,53800.000000,0,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Gym / Fitness Center,Café,Hotel,Bar,Pizza Place
2,EC1M,Camden,51.520813,-0.106501,50000.000000,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
3,EC1M,City of London,51.520806,-0.098285,65400.000000,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
4,EC1M,Islington,51.521362,-0.102184,53800.000000,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,WC2H,Westminster,51.511539,-0.128543,58849.397590,2,Coffee Shop,Theater,Pub,Bakery,Cocktail Bar,Ice Cream Shop,Hotel,Steakhouse,Sushi Restaurant,Gym
86,WC2N,Islington,51.524565,-0.112042,53800.000000,2,Pub,Hotel,Theater,Garden,Bakery,Coffee Shop,Cocktail Bar,Burger Joint,Ice Cream Shop,Pizza Place
87,WC2N,Westminster,51.508958,-0.124997,58939.189189,2,Pub,Hotel,Theater,Garden,Bakery,Coffee Shop,Cocktail Bar,Burger Joint,Ice Cream Shop,Pizza Place
88,WC2R,Islington,51.524565,-0.112042,53800.000000,2,Pub,Coffee Shop,Sandwich Place,Cocktail Bar,Burger Joint,Restaurant,Bar,Hotel,Theater,Pizza Place


In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Ln_merged['Latitude'], Ln_merged['Longitude'], Ln_merged['Postcode District'], Ln_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

With our clusters set and visulaised, we can now take a look at each one individually. We can use the value_count operation on each cluster dataframe to find the total number of each venue in each cluster

In [26]:
cl1 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 0, Ln_merged.columns[[0,4] + list(range(5, Ln_merged.shape[1]))]]
cl1 = cl1.round()
cl1

,Postcode District,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EC1A,65400.0,0,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Gym / Fitness Center,Café,Hotel,Bar,Pizza Place
1,EC1A,53800.0,0,Pub,Coffee Shop,Italian Restaurant,French Restaurant,Sandwich Place,Gym / Fitness Center,Café,Hotel,Bar,Pizza Place
5,EC1N,50010.0,0,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Falafel Restaurant,Sushi Restaurant
6,EC1N,65400.0,0,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Falafel Restaurant,Sushi Restaurant
7,EC1N,53800.0,0,Pub,Coffee Shop,Sandwich Place,Hotel,Italian Restaurant,Food Truck,Cocktail Bar,Wine Bar,Falafel Restaurant,Sushi Restaurant
8,EC1P,53800.0,0,Pub,Coffee Shop,Pizza Place,Cocktail Bar,Sandwich Place,Garden,Middle Eastern Restaurant,History Museum,Food Truck,Bar
9,EC1R,50000.0,0,Pub,Coffee Shop,Café,Food Truck,Bar,Pizza Place,Vietnamese Restaurant,Middle Eastern Restaurant,Bakery,Sandwich Place
10,EC1R,53282.0,0,Pub,Coffee Shop,Café,Food Truck,Bar,Pizza Place,Vietnamese Restaurant,Middle Eastern Restaurant,Bakery,Sandwich Place
44,EC4A,65400.0,0,Pub,Coffee Shop,Sandwich Place,Italian Restaurant,Wine Bar,Restaurant,Hotel,French Restaurant,Burrito Place,Sushi Restaurant
45,EC4A,53800.0,0,Pub,Coffee Shop,Sandwich Place,Italian Restaurant,Wine Bar,Restaurant,Hotel,French Restaurant,Burrito Place,Sushi Restaurant


In [27]:
cl2 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 1, Ln_merged.columns[[0,4] + list(range(5, Ln_merged.shape[1]))]]
cl2 = cl2.round()
cl2

,Postcode District,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,EC1M,50000.0,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
3,EC1M,65400.0,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
4,EC1M,53800.0,1,Pub,Coffee Shop,Food Truck,Hotel,Gym / Fitness Center,Café,French Restaurant,Italian Restaurant,Beer Bar,Wine Bar
11,EC1V,54203.0,1,Coffee Shop,Pub,Italian Restaurant,Café,Bar,Sandwich Place,Cocktail Bar,Vietnamese Restaurant,Food Truck,Gym / Fitness Center
12,EC1V,52259.0,1,Coffee Shop,Pub,Italian Restaurant,Café,Bar,Sandwich Place,Cocktail Bar,Vietnamese Restaurant,Food Truck,Gym / Fitness Center
15,EC2A,65400.0,1,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Italian Restaurant,Pub,Pizza Place,Bar,Café,Cocktail Bar
16,EC2A,54300.0,1,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Italian Restaurant,Pub,Pizza Place,Bar,Café,Cocktail Bar
17,EC2A,50757.0,1,Coffee Shop,Gym / Fitness Center,Food Truck,Hotel,Italian Restaurant,Pub,Pizza Place,Bar,Café,Cocktail Bar
18,EC2M,65400.0,1,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Bar,Café,Plaza,French Restaurant
19,EC2M,54300.0,1,Pub,Hotel,Coffee Shop,Food Truck,Pizza Place,Gym / Fitness Center,Bar,Café,Plaza,French Restaurant


In [28]:
cl3 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 2, Ln_merged.columns[[0,4] + list(range(5, Ln_merged.shape[1]))]]
cl3 = cl3.round()
cl3

,Postcode District,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,WC1A,50200.0,2,Pub,Coffee Shop,Hotel,Theater,Café,Italian Restaurant,Pizza Place,Sandwich Place,Garden,Exhibit
60,WC1A,53800.0,2,Pub,Coffee Shop,Hotel,Theater,Café,Italian Restaurant,Pizza Place,Sandwich Place,Garden,Exhibit
61,WC1B,49845.0,2,Pub,Coffee Shop,Hotel,Sandwich Place,Café,Pizza Place,Cocktail Bar,Bar,Theater,Bookstore
62,WC1B,53800.0,2,Pub,Coffee Shop,Hotel,Sandwich Place,Café,Pizza Place,Cocktail Bar,Bar,Theater,Bookstore
79,WC2B,50200.0,2,Pub,Theater,Coffee Shop,Hotel,Burger Joint,Bakery,Clothing Store,Restaurant,Japanese Restaurant,Café
80,WC2B,53800.0,2,Pub,Theater,Coffee Shop,Hotel,Burger Joint,Bakery,Clothing Store,Restaurant,Japanese Restaurant,Café
81,WC2B,59032.0,2,Pub,Theater,Coffee Shop,Hotel,Burger Joint,Bakery,Clothing Store,Restaurant,Japanese Restaurant,Café
82,WC2E,59000.0,2,Coffee Shop,Theater,Ice Cream Shop,Burger Joint,Bakery,Cocktail Bar,Hotel,Clothing Store,Steakhouse,Gym
83,WC2H,50200.0,2,Coffee Shop,Theater,Pub,Bakery,Cocktail Bar,Ice Cream Shop,Hotel,Steakhouse,Sushi Restaurant,Gym
84,WC2H,53800.0,2,Coffee Shop,Theater,Pub,Bakery,Cocktail Bar,Ice Cream Shop,Hotel,Steakhouse,Sushi Restaurant,Gym


In [29]:
cl4 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 3, Ln_merged.columns[[0,4] + list(range(5, Ln_merged.shape[1]))]]
cl4 = cl4.round()
cl4

,Postcode District,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,EC3M,65400.0,3,Hotel,Restaurant,Gym / Fitness Center,Cocktail Bar,South American Restaurant,Garden,Salad Place,Beer Bar,Hotel Bar,Portuguese Restaurant


In [30]:
cl5 = Ln_merged.loc[Ln_merged['Cluster Labels'] == 4, Ln_merged.columns[[0,4] + list(range(5, Ln_merged.shape[1]))]]
cl5 = cl5.round()
cl5

,Postcode District,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,EC1Y,65400.0,4,Coffee Shop,Gym / Fitness Center,Food Truck,Italian Restaurant,Café,Pub,Concert Hall,Bar,Art Gallery,Sushi Restaurant
14,EC1Y,49321.0,4,Coffee Shop,Gym / Fitness Center,Food Truck,Italian Restaurant,Café,Pub,Concert Hall,Bar,Art Gallery,Sushi Restaurant
30,EC2Y,65400.0,4,Coffee Shop,Gym / Fitness Center,Food Truck,Art Gallery,Italian Restaurant,Café,Turkish Restaurant,Concert Hall,Bagel Shop,Beer Bar
31,EC2Y,49989.0,4,Coffee Shop,Gym / Fitness Center,Food Truck,Art Gallery,Italian Restaurant,Café,Turkish Restaurant,Concert Hall,Bagel Shop,Beer Bar


In [31]:
for column in cl1:
    print(cl1[column].value_counts())

WC2A    4
WC1V    3
EC4Y    3
EC4A    3
EC1N    3
EC4M    2
EC1A    2
WC1N    2
WC1X    2
WC1H    2
EC1R    2
EC4V    2
EC4P    1
EC1P    1
Name: Postcode District, dtype: int64
53800.0    12
65400.0     8
59000.0     3
50200.0     2
45278.0     1
50010.0     1
41448.0     1
51885.0     1
47428.0     1
53282.0     1
50000.0     1
Name: Average Income, dtype: int64
0    32
Name: Cluster Labels, dtype: int64
Pub    32
Name: 1st Most Common Venue, dtype: int64
Coffee Shop       30
Sandwich Place     2
Name: 2nd Most Common Venue, dtype: int64
Hotel                 11
Sandwich Place        11
Italian Restaurant     4
Coffee Shop            2
Pizza Place            2
Café                   2
Name: 3rd Most Common Venue, dtype: int64
Italian Restaurant    7
Sandwich Place        7
Hotel                 6
Café                  4
Cocktail Bar          4
French Restaurant     2
Food Truck            2
Name: 4th Most Common Venue, dtype: int64
Italian Restaurant      8
Sandwich Place          6


In [32]:
for column in cl2:
    print(cl2[column].value_counts())

EC2M    4
EC1M    3
EC3N    3
EC2A    3
EC2P    2
EC3R    2
WC1E    2
EC2N    2
EC4R    2
EC3P    2
EC4N    2
EC2R    2
EC2V    2
EC3V    2
EC1V    2
EC3A    2
WC1R    1
Name: Postcode District, dtype: int64
65400.0    14
53800.0    13
54300.0     2
52711.0     1
52259.0     1
54203.0     1
70900.0     1
53100.0     1
50757.0     1
46863.0     1
50198.0     1
50000.0     1
Name: Average Income, dtype: int64
1    38
Name: Cluster Labels, dtype: int64
Pub            19
Coffee Shop    14
Hotel           5
Name: 1st Most Common Venue, dtype: int64
Pub                     14
Hotel                   10
Coffee Shop              9
Gym / Fitness Center     3
Cocktail Bar             2
Name: 2nd Most Common Venue, dtype: int64
Coffee Shop             11
Italian Restaurant       6
Food Truck               6
Gym / Fitness Center     3
Pub                      2
Exhibit                  2
Garden                   2
Sandwich Place           2
Pizza Place              2
Hotel                    2
Nam

In [33]:
for column in cl3:
    print(cl3[column].value_counts())

WC2H    3
WC2B    3
WC1B    2
WC2N    2
WC1A    2
WC2R    2
WC2E    1
Name: Postcode District, dtype: int64
53800.0    6
50200.0    3
59000.0    2
49845.0    1
58939.0    1
58849.0    1
59032.0    1
Name: Average Income, dtype: int64
2    15
Name: Cluster Labels, dtype: int64
Pub            11
Coffee Shop     4
Name: 1st Most Common Venue, dtype: int64
Theater        7
Coffee Shop    6
Hotel          2
Name: 2nd Most Common Venue, dtype: int64
Hotel             4
Pub               3
Coffee Shop       3
Theater           2
Sandwich Place    2
Ice Cream Shop    1
Name: 3rd Most Common Venue, dtype: int64
Bakery            3
Hotel             3
Cocktail Bar      2
Garden            2
Sandwich Place    2
Theater           2
Burger Joint      1
Name: 4th Most Common Venue, dtype: int64
Burger Joint    5
Café            4
Bakery          3
Cocktail Bar    3
Name: 5th Most Common Venue, dtype: int64
Bakery                3
Ice Cream Shop        3
Italian Restaurant    2
Restaurant            

In [34]:
for column in cl4:
    print(cl4[column].value_counts())

EC3M    1
Name: Postcode District, dtype: int64
65400.0    1
Name: Average Income, dtype: int64
3    1
Name: Cluster Labels, dtype: int64
Hotel    1
Name: 1st Most Common Venue, dtype: int64
Restaurant    1
Name: 2nd Most Common Venue, dtype: int64
Gym / Fitness Center    1
Name: 3rd Most Common Venue, dtype: int64
Cocktail Bar    1
Name: 4th Most Common Venue, dtype: int64
South American Restaurant    1
Name: 5th Most Common Venue, dtype: int64
Garden    1
Name: 6th Most Common Venue, dtype: int64
Salad Place    1
Name: 7th Most Common Venue, dtype: int64
Beer Bar    1
Name: 8th Most Common Venue, dtype: int64
Hotel Bar    1
Name: 9th Most Common Venue, dtype: int64
Portuguese Restaurant    1
Name: 10th Most Common Venue, dtype: int64


In [35]:
for column in cl5:
    print(cl5[column].value_counts())

EC2Y    2
EC1Y    2
Name: Postcode District, dtype: int64
65400.0    2
49321.0    1
49989.0    1
Name: Average Income, dtype: int64
4    4
Name: Cluster Labels, dtype: int64
Coffee Shop    4
Name: 1st Most Common Venue, dtype: int64
Gym / Fitness Center    4
Name: 2nd Most Common Venue, dtype: int64
Food Truck    4
Name: 3rd Most Common Venue, dtype: int64
Art Gallery           2
Italian Restaurant    2
Name: 4th Most Common Venue, dtype: int64
Café                  2
Italian Restaurant    2
Name: 5th Most Common Venue, dtype: int64
Pub     2
Café    2
Name: 6th Most Common Venue, dtype: int64
Turkish Restaurant    2
Concert Hall          2
Name: 7th Most Common Venue, dtype: int64
Concert Hall    2
Bar             2
Name: 8th Most Common Venue, dtype: int64
Art Gallery    2
Bagel Shop     2
Name: 9th Most Common Venue, dtype: int64
Sushi Restaurant    2
Beer Bar            2
Name: 10th Most Common Venue, dtype: int64


With the best district from each cluster identified, we can create a new dataframe made up of the five of them, which we can visulise with folium

In [65]:
df3 = Ln_merged[(Ln_merged['Postcode District'] == 'EC4Y')]
df3 = df3.append(Ln_merged[(Ln_merged['Postcode District'] == 'EC3N')])
df3 = df3.append(Ln_merged[(Ln_merged['Postcode District'] == 'WC2H')])
df3 = df3.append(Ln_merged[(Ln_merged['Postcode District'] == 'EC3M')])
df3 = df3.append(Ln_merged[(Ln_merged['Postcode District'] == 'EC2Y')])
df3.drop(columns = "District", inplace = True)
df3.groupby(["Postcode District"]).mean()
df3.drop([57,58,36,37,84,85,31], inplace = True)
d = {'colour': ["red", "blue", "green", "yellow", "purple"], "Postcode District":["EC4Y", "EC3N", "WC2H", "EC3M", "EC2Y"]}
df4 = pd.DataFrame(data=d)
df3 = df3.merge(df4)
df3

,Postcode District,Latitude,Longitude,Average Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,colour
0,EC4Y,51.512879,-0.108688,65400.0,0,Pub,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Restaurant,History Museum,Bar,Cocktail Bar,Garden,red
1,EC3N,51.512033,-0.076553,65400.0,1,Hotel,Pub,Coffee Shop,Garden,Gym / Fitness Center,Cocktail Bar,French Restaurant,History Museum,Pizza Place,Scenic Lookout,blue
2,WC2H,51.514658,-0.126763,50200.0,2,Coffee Shop,Theater,Pub,Bakery,Cocktail Bar,Ice Cream Shop,Hotel,Steakhouse,Sushi Restaurant,Gym,green
3,EC3M,51.511831,-0.082165,65400.0,3,Hotel,Restaurant,Gym / Fitness Center,Cocktail Bar,South American Restaurant,Garden,Salad Place,Beer Bar,Hotel Bar,Portuguese Restaurant,yellow
4,EC2Y,51.519254,-0.093359,65400.0,4,Coffee Shop,Gym / Fitness Center,Food Truck,Art Gallery,Italian Restaurant,Café,Turkish Restaurant,Concert Hall,Bagel Shop,Beer Bar,purple


In [66]:
map_final = folium.Map(location=[latitude, longitude], zoom_start=13)


for lat, lng, postcode, pin in zip(df3['Latitude'], df3['Longitude'], df3['Postcode District'], df3["colour"]): 
    label = '{}'.format(postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color=pin,
        fill=True,
        fill_color=pin,
        fill_opacity=0.7,
        parse_html=False).add_to(map_final)  
    
map_final

In [67]:
df3.drop([2,3], inplace = True)
map_final2 = folium.Map(location=[latitude, longitude], zoom_start=13)


for lat, lng, postcode, pin in zip(df3['Latitude'], df3['Longitude'], df3['Postcode District'], df3["colour"]): 
    label = '{}'.format(postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color=pin,
        fill=True,
        fill_color=pin,
        fill_opacity=0.7,
        parse_html=False).add_to(map_final2)  
    
map_final2